<a href="https://colab.research.google.com/github/rdiazrincon/SLEAP_workshop/blob/master/Training_and_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows you how to run training and inference on your own SLEAP dataset using the command-line interface.

You'll use [Google Drive](https://www.google.com/drive) for moving your data to and from Colab. We'll guide you through the process.

# Installation

Before we install SLEAP we need to set Colab to use TensorFlow 2.

In [1]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


Let's confirm that we have a GPU available. This next line should return something like "/device:GPU:0". If you instead see an empty string as the result, go to "Notebook settings" in the "Edit" menu and select "GPU" as the hardware accelerator.

In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

Now let's use `pip` to install SLEAP from PyPI.

Note: This installation method should also work on other Linux machines, such as an HPC cluster, or on any system where you aren't planning to use a GPU. To use a GPU on a Windows machine you'll need to install using `conda`.

In [3]:
!pip install sleap==1.2.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Getting your training data into Colab

You'll need to get your training data into Colab. So let's get the data from the workshop's [repo](https://github.com/rdiazrincon/SLEAP_workshop)

The training package contains both labeled data as well as the labeled images which will be used for training. One advantage to training packages is that it doesn't depend on paths to other files (i.e., videos) to be messed up when you copy your project to another volume.

See [this guide](https://sleap.ai/guides/training-package.html) for exporting a training package from SLEAP.


The following cell sets your current working directory to the directory with your training package. This will ensure that the output from training (i.e., the models) and from interence (i.e., predictions) will all be saved in this directory.

**Important**: For this demo, I'll assume you have the file `data.pkg.slp`. If you placed your training pckage somewhere else, you'll need to adjust the path below.

In [6]:
!wget -O video.AVI https://github.com/rdiazrincon/SLEAP_Data/blob/master/videos/video.AVI?raw=true
!wget -O data.pkg.slp https://github.com/rdiazrincon/SLEAP_workshop/blob/master/examples/data.pkg.slp?raw=true
!wget -O baseline_medium_rf.bottomup.json https://github.com/rdiazrincon/SLEAP_Data/blob/master/training_profiles/baseline_medium_rf.bottomup.json?raw=true
!wget -O baseline_medium_rf.topdown.json https://github.com/rdiazrincon/SLEAP_Data/blob/master/training_profiles/baseline_medium_rf.topdown.json?raw=true
!wget -O baseline.centroid.json https://github.com/rdiazrincon/SLEAP_Data/blob/master/training_profiles/baseline.centroid.json?raw=true

--2022-08-02 20:44:13--  https://github.com/rdiazrincon/SLEAP_Data/blob/master/videos/video.AVI?raw=true
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/rdiazrincon/SLEAP_Data/raw/master/videos/video.AVI [following]
--2022-08-02 20:44:13--  https://github.com/rdiazrincon/SLEAP_Data/raw/master/videos/video.AVI
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/rdiazrincon/SLEAP_Data/master/videos/video.AVI [following]
--2022-08-02 20:44:13--  https://media.githubusercontent.com/media/rdiazrincon/SLEAP_Data/master/videos/video.AVI
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:4

## Training

Now you're ready to train a model! We'll use the command-line interface for training, and train a model for confidence maps using the default **training profile**. The training profile determines the model architecture, the learning rate for training, and other training hyperparameters.

When you start running this cell, you'll see the training parameters listed and then you'll see the training and validation loss for each training epoch.

If you're happy with the validation loss you see for an epoch during training, you're welcome to stop training by clicking the stop button next to the notebook cell running training. The version of the model with the lowest validation loss is saved during training, and that's what will be used for inference. If you don't stop training, it will run for 200 epochs, or until validation loss fails to improve for some number of epochs (controlled by the `early_stopping` parameter in the training profile).

**Important**: If your training package isn't named `colab.pkg.slp`, you'll need to adjust the name below.

In [7]:
!sleap-train baseline_medium_rf.bottomup.json data.pkg.slp

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:sleap.nn.training:Versions:
SLEAP: 1.2.6
TensorFlow: 2.8.2
Numpy: 1.21.5
Python: 3.7.13
OS: Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
INFO:sleap.nn.training:Training labels file: data.pkg.slp
INFO:sleap.nn.training:Training profile: baseline_medium_rf.bottomup.json
INFO:sleap.nn.training:
INFO:sleap.nn.training:Arguments:
INFO:sleap.nn.training:{
    "training_job_path": "baseline_medium_rf.bottomup.json",
    "labels_path": "data.pkg.slp",
    "video_paths": [
        ""
    ],
    "val_labels": null,
    "test_labels": null,
    "tensorboard": false,
    "save_viz": false,
    "zmq": false,
    "run_name": "",
    "prefix": "",
    "suffix": "",
    "cpu": false,
    "first_gpu": false,
    "last_gpu": false,
    "gpu": 0
}
INFO:sleap.nn.training:
INFO:sleap.nn.training:Training job:
INFO:sleap.nn.training:{
    "data": {
        "labels": {
            "training_labels": null,
            "validation_labels": null,
      

Once training finishes, you'll have a trained model for confidence maps on your Google Drive. There will be a `models/` directory inside your `sleap/` directory (or wherever you had the training package), and inside this there will be a `colab_demo.bottomup` directory (the name of this directory was set by the `--run_name` parameter). 

This `colab_demo.bottomup` directory contains all the files SLEAP needs to use this model. You can copy it to a local drive if you want to use it for running inference from the SLEAP GUI, copy it to a network drive if you want to run inference from an HPC cluster, or leave it on your Google Drive if you want to run inference on Colab... as we'll do below.

## Training other models

The **bottomup** model you trained above can be used for "bottom up" inference. You can also train a **centroid** model and a **centered instance** model for "top down" inference. See [here](https://sleap.ai/#getting-started-with-sleap) for more information about these two different approaches.

Here's how to train centroid and centered instance models using the default training settings:

In [ ]:
!sleap-train baseline_medium_rf.topdown.json data.pkg.slp

In [ ]:
!sleap-train baseline.centroid.json data.pkg.slp

# Inference

At this point you should have SLEAP installed, your Google Drive mounted, and trained models saved on your Google Drive. If you've been working through the notebook, you should have a `models` subdirectory inside your current working directory. Let's take a look:

In [8]:
!ls models

baseline_medium_rf.bottomup


We'll also need a video for which we want predictions. Copy the video onto your Google drive.

For this demo we'll just get predictions for the first 200 frames (or you can adjust the `--frames` parameter below or remove it to run on the whole video).

**Important**: If your video is not named `colab_demo.mp4`, change this in the following cell to match the name of your video. If you trained top-down models and not a bottom-up model, see the end of the notebook for how to run inference with the pair of top-down models.

In [11]:
!sleap-track video.AVI --frames 0-200 --tracking.tracker flow --tracking.similarity centroid --tracking.match greedy --tracking.clean_instance_count 2 --tracking.target_instance_count 2 -m models/baseline_medium_rf.bottomup

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Started inference at: 2022-08-02 21:13:29.884276
Args:
{
│   'data_path': 'video.AVI',
│   'models': ['models/baseline_medium_rf.bottomup'],
│   'frames': '0-200',
│   'only_labeled_frames': False,
│   'only_suggested_frames': False,
│   'output': None,
│   'no_empty_frames': False,
│   'verbosity': 'rich',
│   'video.dataset': None,
│   'video.input_format': 'channels_last',
│   'cpu': False,
│   'first_gpu': False,
│   'last_gpu': False,
│   'gpu': 0,
│   'max_edge_length_ratio': 0.25,
│   'dist_penalty_weight': 1.0,
│   'batch_size': 4,
│   'open_in_gui': False,
│   'peak_threshold': 0.2,
│   'tracking.tracker': 'flow',
│   'tracking.target_instance_count': 2,
│   'tracking.pre_cull_to_target': None,
│   'tracking.pre_cull_iou_threshold': None,
│   'tracking.post_connect_single_breaks': None,
│   'tracking.clean_instance_count': 2,
│   'tracking.clean_iou_threshold': None,
│   'tracking.similarity': 'centroid',
│   'tracking.match'

When inference is finished, it will save the predictions in a file which can be opened in the GUI as a SLEAP project file. The file will be in the same directory as the video and the filename will be `{video filename}.predictions.slp`.

Let's inspect the predictions file:

In [12]:
!sleap-inspect video.AVI.predictions.slp

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Labeled frames: 201
Tracks: 2
Video files:
  video.AVI
    labeled frames: 201
    labeled frames from 0 to 200
    user labeled frames: 0
    tracks: 2
    max instances in frame: 2
Total user labeled frames: 0

Provenance:
  sleap_version: 1.2.6
  platform: Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
  command: /usr/local/bin/sleap-track video.AVI --frames 0-200 --tracking.tracker flow --tracking.similarity centroid --tracking.match greedy --tracking.clean_instance_count 2 --tracking.target_instance_count 2 -m models/baseline_medium_rf.bottomup
  data_path: video.AVI
  model_paths: ['models/baseline_medium_rf.bottomup/training_config.json']
  output_path: video.AVI.predictions.slp
  predictor: BottomUpPredictor
  total_elapsed: 43.442108392715454
  start_timestamp: 2022-08-02 21:13:29.884276
  finish_timestamp: 2022-08-02 21:14:13.326364
  args: {'data_path': 'video.AVI', 'models': ['models/baseline_medium_rf.bottomup'], 'frames'

You can copy this file from your Google Drive to a local drive and open it in the SLEAP GUI app (or open it directly if you have your Google Drive mounted on your local machine). If the video is in the same directory as the predictions file, SLEAP will automatically find it; otherwise, you'll be prompted to locate the video (since the path to the video on your local machine will be different than the path to the video on Colab).

In [13]:
!sleap-render video.AVI.predictions.slp --frames 0-200

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Saving config: /root/.sleap/1.2.6/preferences.yaml
Writing video with 201 frame images...
INFO:sleap.io.visuals:Chunks: 4, chunk size: 64
Finished 64 frames in 2.127247532000183 s, fps = 30.085826419938464, approx 4.553639248187892 s remaining
Finished 128 frames in 2.434497031999854 s, fps = 52.577595420131814, approx 1.3884240885624166 s remaining
Finished 192 frames in 2.7427903379998497 s, fps = 70.00170495715467, approx 0.12856829709374296 s remaining
Finished 201 frames in 2.788367085000118 s, fps = 72.0852003601927, approx 0.0 s remaining
Done in 2.8240254259999347 s, fps = 71.17499656676415.
Video saved as: video.AVI.predictions.slp.avi


## Inference parameters

One important option when running inference is whether (and how) you want to track instance identities. If you omit `--tracking.tracker flow` then the identities will not be tracked. Tracking methods/options are explained [here](https://sleap.ai/guides/proofreading.html#tracking-methods).

You can see all of the command-line arguments by calling `sleap-track` with the `--help` argument, like so:

In [ ]:
!sleap-track --help

## Inference with top-down models

If you trained the pair of models needed for top-down inference, you can call `sleap-track` with `-m path/to/model` for each model, like so:

In [ ]:
!sleap-track colab_demo.video.mp4 \
    --frames 0-200 \
    --tracking.tracker flow \
    --tracking.similarity centroid \
    --tracking.match greedy \
    --tracking.clean_instance_count 2 \
    -m models/baseline_medium_rf.topdown \
    -m models/baseline.centroid